# **Quick cumcount**

# Contexte et motivation

Sprint de recherche de performance motivé dans par l'analyse exploratoire des deux variables `SK` de la table `previous_application`.

Le produit de cette recherche sont les fonctions `pepper.np_utils.subindex` et `pepper.pd_utils.subindex`.

# Spécification
Exploitons à bon escient notre hypothèse sur l'ordre temporel des SK : un groupby est coûteux.

Or ici, il est plus pertinent de :
1. trier le tableau par client, puis demandes précédentes
2. attribuer un indice relatif pour chaque demande précédente, en ordre inverse (0 pour la dernière, 1 pour l'avant dernière, etc)
3. remplacer le code de statut par sa version courte
4. pivoter pour mettre les demandes précédentes en colonne
5. produire une colonne de synthèse des profils
6. produire une distribution en classes
7. trier le tableau cf. ces classes
8. en donner un visuel à l'aide d'une heatmap

# Le cas d'utilisation

In [6]:
from home_credit.load import get_previous_application
from home_credit.utils import get_datablock
prev_app = get_previous_application()
sk_status = get_datablock(prev_app, "SK_|NAME_CONTRACT_STATUS")
display(sk_status)

previous_application,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_STATUS
0,2030495,271877,Approved
1,2802425,108129,Approved
2,2523466,122040,Approved
3,2819243,176158,Approved
4,1784265,202054,Refused
...,...,...,...
1670209,2300464,352015,Approved
1670210,2357031,334635,Approved
1670211,2659632,249544,Approved
1670212,2785582,400317,Approved


In [7]:
sk_status = sk_status[["SK_ID_CURR", "SK_ID_PREV", "NAME_CONTRACT_STATUS"]]
sorted_sk_status = sk_status.sort_values(by=["SK_ID_CURR", "SK_ID_PREV"])
display(sorted_sk_status)

previous_application,SK_ID_CURR,SK_ID_PREV,NAME_CONTRACT_STATUS
201668,100001,1369693,Approved
892077,100002,1038818,Approved
575941,100003,1810518,Approved
1223745,100003,2396755,Approved
1021650,100003,2636178,Approved
...,...,...,...
729432,456255,1708056,Refused
214743,456255,1743609,Approved
608510,456255,2073384,Approved
1383554,456255,2631384,Approved


# Inputs

In [8]:
import numpy as np
a = np.array([0, 0, 1, 1, 1, 3, 5, 5, 11, 11, 11, 11])
b_series = sorted_sk_status.SK_ID_CURR
b = b_series.values

# Commons

In [2]:
import numpy as np
import pandas as pd

def pd_wrapper_of_np_soluce(soluce_f, s):
    p = soluce_f(s.values)
    df = s.reset_index()
    df.insert(2, "POS", p)
    df.set_index("index", inplace=True)
    df.index.name = None
    return df

# Solutions

Notons que la solution 1 n'est pas la première à avoir été trouvée.

Il est probable que sinon, cette investigation n'aurait pas été menée.

In [10]:
def soluce_1(s) -> pd.Series:
    return s.groupby(s).cumcount()

def soluce_1_np(a) -> np.ndarray:
    return soluce_1(pd.Series(a)).to_numpy()

def soluce_1_pd(s) -> pd.DataFrame:
    p = soluce_1(s).rename("POS")
    return pd.concat([s, p], axis=1)

print(soluce_1_np(a))
display(soluce_1_pd(b_series))

[0 1 0 1 2 0 0 1 0 1 2 3]


,SK_ID_CURR,POS
201668,100001,0
892077,100002,0
575941,100003,0
1223745,100003,1
1021650,100003,2
...,...,...
729432,456255,3
214743,456255,4
608510,456255,5
1383554,456255,6


In [ ]:
def soluce_2(a):
    indices = np.concatenate((
        [0],
        np.where(np.diff(a) != 0)[0] + 1,
        [len(a)]
    ))
    return np.concatenate([
        np.arange(0, len(arr_section))
        for arr_section in np.split(a, indices) if len(arr_section) > 0])

def soluce_2_np(a):
    return soluce_2(a)

def soluce_2_pd(s):
    return pd_wrapper_of_np_soluce(soluce_2, s)

print(soluce_2_np(a))
display(soluce_2_pd(b_series))

[0 1 0 1 2 0 0 1 0 1 2 3]


,SK_ID_CURR,POS
201668,100001,0
892077,100002,0
575941,100003,0
1223745,100003,1
1021650,100003,2
...,...,...
729432,456255,3
214743,456255,4
608510,456255,5
1383554,456255,6


In [ ]:
def soluce_3(a):
    _, c = np.unique(a, return_counts=True)
    rows = [list(range(c_k)) for c_k in c]
    return np.concatenate(rows, axis=0)

def soluce_3_np(a):
    return soluce_3(a)

def soluce_3_pd(s):
    return pd_wrapper_of_np_soluce(soluce_3, s)

print(soluce_3(a))
display(soluce_3_pd(b_series))

[0 1 0 1 2 0 0 1 0 1 2 3]


,SK_ID_CURR,POS
201668,100001,0
892077,100002,0
575941,100003,0
1223745,100003,1
1021650,100003,2
...,...,...
729432,456255,3
214743,456255,4
608510,456255,5
1383554,456255,6


Cette solution surperforme le `groupby.cumcount` de pandas (voir benchmark ci-dessous), et je pense pouvoir encore l'améliorer, il y a des optimisations algorithmiques possibles (*c > k+1 est un sous-ensemble de c > k*).

In [ ]:
def soluce_4(a):
    # Find unique values, their indices, and their counts
    _, i, c = np.unique(a, return_index=True, return_counts=True)
    x = np.zeros_like(a)
    for k in range(1, np.max(c)):
        x[i[c > k] + k] = k
    return x

def soluce_4_np(a):
    return soluce_4(a)

def soluce_4_pd(s):
    return pd_wrapper_of_np_soluce(soluce_4, s)

print(soluce_4(a))
display(soluce_4_pd(b_series))

[0 1 0 1 2 0 0 1 0 1 2 3]


,SK_ID_CURR,POS
201668,100001,0
892077,100002,0
575941,100003,0
1223745,100003,1
1021650,100003,2
...,...,...
729432,456255,3
214743,456255,4
608510,456255,5
1383554,456255,6


In [ ]:
def soluce_5(a):
    # Find unique values, their indices, and their counts
    _, c = np.unique(a, return_counts=True)

    # Create an array [0, 1, ..., max(c)-1]
    p = np.arange(np.max(c))

    # Generate a boolean matrix indicating if c > p and multiply it by p
    x = (c.reshape(-1, 1) > p) * p

    # Recode 0 as -1
    x[x == 0] = -1

    # Recode the first column with 0
    x[:, 0] = 0

    # Flatten the array
    y = x.ravel()

    # Return a subarray containing only elements greater than -1
    return y[y > -1]

def soluce_5_np(a):
    return soluce_5(a)

def soluce_5_pd(s):
    return pd_wrapper_of_np_soluce(soluce_5, s)

print(soluce_5(a))
display(soluce_5_pd(b_series))

[0 1 0 1 2 0 0 1 0 1 2 3]


,SK_ID_CURR,POS
201668,100001,0
892077,100002,0
575941,100003,0
1223745,100003,1
1021650,100003,2
...,...,...
729432,456255,3
214743,456255,4
608510,456255,5
1383554,456255,6


In [ ]:
def soluce_6(a):
    # Find unique values, their indices, and their counts
    _, i, c = np.unique(a, return_index=True, return_counts=True)

    # Create an array [0, 1, ..., max(c)-1]
    p = np.arange(1, np.max(c))

    # Generate a boolean matrix indicating if c > p
    c_gt_p = (c.reshape(-1, 1) > p).T

    x = np.zeros_like(a)
    for k in p:
        x[i[c_gt_p[k-1]] + k] = k
    
    return x     

def soluce_6_np(a):
    return soluce_6(a)

def soluce_6_pd(s):
    return pd_wrapper_of_np_soluce(soluce_6, s)

print(soluce_6(a))
display(soluce_6_pd(b_series))

[0 1 0 1 2 0 0 1 0 1 2 3]


,SK_ID_CURR,POS
201668,100001,0
892077,100002,0
575941,100003,0
1223745,100003,1
1021650,100003,2
...,...,...
729432,456255,3
214743,456255,4
608510,456255,5
1383554,456255,6


# Benchmark

In [ ]:
import timeit

def benchmark(n_iter=10):
    np_fs = [soluce_1_np, soluce_2_np, soluce_3_np, soluce_4_np, soluce_5_np, soluce_6_np]
    pd_fs = [soluce_1_pd, soluce_2_pd, soluce_3_pd, soluce_4_pd, soluce_5_pd, soluce_6_pd]

    # Ident
    res_a_np_ref = soluce_1_np(a)
    res_a_pd_ref = soluce_1_pd(pd.Series(a))
    res_b_np_ref = soluce_1_np(b_series.values)
    res_b_pd_ref = soluce_1_pd(b_series)

    for np_f, pd_f in zip(np_fs, pd_fs):
        a_series = pd.Series(a)
        b = b_series.values
        print(f"ident(a) for {np_f.__name__}: {(np_f(a) == res_a_np_ref).all()}")
        print(f"ident(a) for {pd_f.__name__}: {(pd_f(a_series) == res_a_pd_ref).all().all()}")
        print(f"ident(b) for {np_f.__name__}: {(np_f(b) == res_b_np_ref).all()}")
        print(f"ident(b) for {pd_f.__name__}: {(pd_f(b_series) == res_b_pd_ref).all().all()}")

    # Perf
    for np_f, pd_f in zip(np_fs, pd_fs):
        a_series = pd.Series(a)
        b = b_series.values
        print(f"perf(a) for {np_f.__name__}: {timeit.timeit(lambda: np_f(a), number=n_iter)} s")
        print(f"perf(a) for {pd_f.__name__}: {timeit.timeit(lambda: pd_f(a_series), number=n_iter)} s")
        print(f"perf(b) for {np_f.__name__}: {timeit.timeit(lambda: np_f(b), number=n_iter)} s")
        print(f"perf(b) for {pd_f.__name__}: {timeit.timeit(lambda: pd_f(b_series), number=n_iter)} s")


benchmark()

ident(a) for soluce_1_np: True
ident(a) for soluce_1_pd: True
ident(b) for soluce_1_np: True
ident(b) for soluce_1_pd: True
ident(a) for soluce_2_np: True
ident(a) for soluce_2_pd: True
ident(b) for soluce_2_np: True
ident(b) for soluce_2_pd: True
ident(a) for soluce_3_np: True
ident(a) for soluce_3_pd: True
ident(b) for soluce_3_np: True
ident(b) for soluce_3_pd: True
ident(a) for soluce_4_np: True
ident(a) for soluce_4_pd: True
ident(b) for soluce_4_np: True
ident(b) for soluce_4_pd: True
ident(a) for soluce_5_np: True
ident(a) for soluce_5_pd: True
ident(b) for soluce_5_np: True
ident(b) for soluce_5_pd: True
ident(a) for soluce_6_np: True
ident(a) for soluce_6_pd: True
ident(b) for soluce_6_np: True
ident(b) for soluce_6_pd: True
perf(a) for soluce_1_np: 0.003748999999515945 s
perf(a) for soluce_1_pd: 0.006676800000604999 s
perf(b) for soluce_1_np: 1.4838340999995125 s
perf(b) for soluce_1_pd: 1.586767300000247 s
perf(a) for soluce_2_np: 0.0007848999994166661 s
perf(a) for soluce_2

# Subindex

In [2]:
from pepper.np_utils import subindex
import numpy as np

a = np.array([0, 0, 1, 1, 1, 3, 5, 5, 11, 11, 11, 11])

# Cas trié mais non indiqué (moindre perf)
i = subindex(a)
display(a)
display(i)

# Cas trié mais indiqué (meilleure perf (vérifier avec timeit))
i = subindex(a, sorted=True)
display(a)
display(i)

# Cas non trié, ce qui est considéré par défaut
np.random.shuffle(a)
i = subindex(a)
display(a)
display(i)

# Cas non trié et mensonge qui ne paie pas : résultat incohérent
np.random.shuffle(a)
i = subindex(a, sorted=True)
display(a)
display(i)


array([ 0,  0,  1,  1,  1,  3,  5,  5, 11, 11, 11, 11])

array([0, 1, 0, 1, 2, 0, 0, 1, 0, 1, 2, 3])

array([ 0,  0,  1,  1,  1,  3,  5,  5, 11, 11, 11, 11])

array([0, 1, 0, 1, 2, 0, 0, 1, 0, 1, 2, 3])

array([ 5,  5,  1, 11,  0,  0,  1, 11,  1, 11,  3, 11])

array([0, 1, 0, 0, 0, 1, 1, 1, 2, 2, 0, 3])

array([11, 11,  1,  5,  1,  0,  0,  3, 11, 11,  5,  1])

array([0, 1, 2, 3, 2, 0, 1, 0, 0, 0, 0, 0])

In [11]:
from home_credit.load import get_previous_application
from home_credit.utils import get_datablock
from pepper.np_utils import subindex as npsi
from pepper.pd_utils import subindex as pdsi

prev_app = get_previous_application()
sk_status = get_datablock(prev_app, "SK_|NAME_CONTRACT_STATUS")
sk_status = sk_status[["SK_ID_CURR", "SK_ID_PREV", "NAME_CONTRACT_STATUS"]]
sorted_sk_status = sk_status.sort_values(by=["SK_ID_CURR", "SK_ID_PREV"])

display(npsi(sorted_sk_status.SK_ID_CURR.to_numpy(), sorted=True))
display(pdsi(sorted_sk_status.SK_ID_CURR, sorted=True))

array([0, 0, 0, ..., 5, 6, 7], dtype=int64)

,SK_ID_CURR,subindex
201668,100001,0
892077,100002,0
575941,100003,0
1223745,100003,1
1021650,100003,2
...,...,...
729432,456255,3
214743,456255,4
608510,456255,5
1383554,456255,6
